# MENA Analysis: Relative Wealth Index (RWI)

### Load packages and setup

In [1]:
import pandas as pd
import geopandas as gpd
import pandas as pd
import numpy as np
import glob

In [2]:
# Define directories
dir_shp = 'M:/MENA/GEO/Boundaries/'
dir_in = 'M:/MENA/GEO/RWI/raw/'
dir_out = 'M:/MENA/GEO/RWI/final/'


# Define output FILE
VAR = 'RWI'
FILE_OUT = 'ADM2_' + VAR + '.csv'

In [3]:
%matplotlib inline

### Load shapefile

In [4]:
# Load shapefile
MENA_shp = gpd.read_file(dir_shp + 'MENA_ADM2.shp')

MENA_shp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Load RWI data

In [5]:
RWI_files = []
for FILE in glob.glob(dir_in + '*.csv'):
    RWI_files.append(FILE)

RWI_full = pd.concat((pd.read_csv(FILE) for FILE in RWI_files), ignore_index=True)
RWI_full

,quadkey,latitude,longitude,rwi,error
0,1.221300e+13,31.868227,36.177979,-0.424,0.506
1,1.221300e+13,30.779598,36.046143,0.124,0.421
2,1.221300e+13,30.002516,35.496826,0.651,0.502
3,1.221122e+13,32.296419,36.156006,0.018,0.533
4,1.221300e+13,31.905541,35.716553,-0.126,0.518
...,...,...,...,...,...
119864,NaN,34.732584,9.261475,-0.296,0.315
119865,NaN,33.091541,11.151123,-0.092,0.345
119866,NaN,35.897950,9.459229,-0.322,0.401
119867,NaN,36.853252,9.569092,-0.560,0.452


In [6]:
# Subset
RWI_full = RWI_full[['longitude', 'latitude', 'rwi', 'error']]

# Convert RWI df to shp
RWI_full = gpd.GeoDataFrame(RWI_full, 
                            geometry = gpd.points_from_xy(RWI_full.longitude, 
                                                          RWI_full.latitude, crs = "EPSG:4326"))

RWI_df = RWI_full.sjoin(MENA_shp, how = "left").groupby(['ID_ADM']).agg({"rwi": ['mean', 'median', 'std',
                                                                                 'min', 'max', 'count']})

RWI_df.reset_index(inplace = True)
RWI_df

ID_ADM       rwi                                      
                   mean  median       std    min    max count
0     15272.0 -0.372449 -0.4190  0.292299 -1.025  0.725    78
1     15273.0 -0.463659 -0.4870  0.233556 -0.869  0.343    41
2     15274.0 -0.341872 -0.4140  0.369730 -0.910  0.904    39
3     15275.0 -0.401233 -0.4095  0.269788 -1.043  0.179    30
4     15276.0  0.116604 -0.0470  0.530185 -0.663  1.483    53
...       ...       ...     ...       ...    ...    ...   ...
1983  65742.0 -0.012810 -0.0590  0.360949 -0.813  0.584    21
1984  65743.0  0.069667  0.1030  0.352913 -0.617  0.713    27
1985  65744.0 -0.129705 -0.1040  0.399903 -1.002  0.701   105
1986  65745.0  0.149700  0.1170  0.401681 -0.713  1.204    90
1987  65746.0  0.004633 -0.0100  0.269869 -0.573  0.575    60

[1988 rows x 7 columns]

In [8]:
RWI_df.to_csv(dir_out + FILE_OUT, index = False)